# 🎯 Thai-HandMate: Demo Integration
### ทดสอบรวม 3 Models: HandC + Face Detection + Typhoon API

---

## 📋 ภาพรวมการทดสอบ

**รายการ Model ที่ทดสอบ:**
- **Model 1:** HandC Gesture Recognition (5 คำ)
- **Model 2:** Face Detection & Emotion Analysis 
- **Model 3:** Typhoon API Sentence Generation (ภาษาไทย)

**เป้าหมาย:**
1. ทดสอบแต่ละ Model แยกกัน
2. รวม 3 Models เข้าด้วยกันเป็น Demo สมบูรณ์
3. สร้าง Workflow ที่ใช้งานง่าย

---

**สัญลักษณ์:** 
- **❌** ล้มเหลว 
- **✅** สำเร็จ 
- **⚙️** กำลังประมวลผล
- **📊** ผลลัพธ์
- **🔧** การตั้งค่า

In [9]:
# 🔧 Cell 1: ตั้งค่าเริ่มต้น - นำเข้า Libraries
print("⚙️ กำลังนำเข้า Libraries...")

import os
import sys
import json
import time
import requests
from datetime import datetime

try:
    import cv2
    print("✅ OpenCV พร้อมใช้งาน")
except ImportError:
    print("❌ OpenCV ไม่พบ - กรุณาติดตั้ง: pip install opencv-python")

try:
    import tensorflow as tf
    print("✅ TensorFlow พร้อมใช้งาน (Version:", tf.__version__, ")")
except ImportError:
    print("❌ TensorFlow ไม่พบ - กรุณาติดตั้ง: pip install tensorflow")

try:
    import mediapipe as mp
    print("✅ MediaPipe พร้อมใช้งาน")
except ImportError:
    print("❌ MediaPipe ไม่พบ - กรุณาติดตั้ง: pip install mediapipe")

print("\n🔧 Libraries Setup เสร็จสิ้น!")
print("="*50)

⚙️ กำลังนำเข้า Libraries...
✅ OpenCV พร้อมใช้งาน


c:\Users\sriha\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\sriha\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\sriha\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

✅ TensorFlow พร้อมใช้งาน (Version: 2.20.0 )
❌ MediaPipe ไม่พบ - กรุณาติดตั้ง: pip install mediapipe

🔧 Libraries Setup เสร็จสิ้น!


In [2]:
# 🔧 Cell 2: ตรวจสอบไฟล์ Model และ Backend
print("⚙️ กำลังตรวจสอบไฟล์ Model...")

try:
    # ตรวจสอบ Backend API
    if os.path.exists("backend/app.py"):
        print("✅ Backend API app.py พบแล้ว")
    else:
        print("❌ Backend API: unknown")
    
    if os.path.exists("backend/.env"):
        print("✅ .env ไฟล์พบแล้ว")
    else:
        print("❌ .env ไฟล์: unknown")
    
    # ตรวจสอบ HandC Model
    if os.path.exists("public/models/handC"):
        print("✅ HandC Model folder พบแล้ว")
        try:
            files = os.listdir("public/models/handC")
            print(f"  📂 ไฟล์ในโฟลเดอร์: {files}")
        except:
            print("  ❌ ไม่สามารถอ่านไฟล์ในโฟลเดอร์")
    else:
        print("❌ HandC Model folder: unknown")
    
    # ตรวจสอบ Face Models
    if os.path.exists("public/face-models"):
        print("✅ Face Models folder พบแล้ว")
        try:
            files = os.listdir("public/face-models")
            print(f"  📂 ไฟล์ในโฟลเดอร์: {len(files)} ไฟล์")
        except:
            print("  ❌ ไม่สามารถอ่านไฟล์ในโฟลเดอร์")
    else:
        print("❌ Face Models folder: unknown")

except Exception as e:
    print(f"❌ ข้อผิดพลาด: {e}")

print("\n🔧 File Check เสร็จสิ้น!")
print("="*50)

⚙️ กำลังตรวจสอบไฟล์ Model...
✅ Backend API app.py พบแล้ว
✅ .env ไฟล์พบแล้ว
✅ HandC Model folder พบแล้ว
  📂 ไฟล์ในโฟลเดอร์: ['handC_test.ipynb', 'metadata.json', 'model.json', 'README.md', 'weights.bin']
✅ Face Models folder พบแล้ว
  📂 ไฟล์ในโฟลเดอร์: 12 ไฟล์

🔧 File Check เสร็จสิ้น!


In [10]:
# 🎯 Cell 3: Model 1 - HandC Gesture Recognition (ทดสอบการโหลด Model)
print("⚙️ กำลังทดสอบการโหลด HandC Model...")

class HandCModelLoader:
    def __init__(self):
        self.vocabulary = []
        self.model_loaded = False
        
        print("🔍 กำลังตรวจสอบไฟล์ HandC Model...")
        
        # อ่าน metadata
        try:
            metadata_path = os.path.join("public", "models", "handC", "metadata.json")
            if os.path.exists(metadata_path):
                with open(metadata_path, 'r', encoding='utf-8') as f:
                    metadata = json.load(f)
                    self.vocabulary = metadata.get('labels', [])
                    print(f"✅ Metadata: พบคำศัพท์ {len(self.vocabulary)} คำ")
                    print(f"📝 คำศัพท์: {self.vocabulary}")
            else:
                print("❌ Metadata: ไม่พบไฟล์")
                return
        except Exception as e:
            print(f"❌ Metadata ผิดพลาด: {e}")
            return
        
        # ตรวจสอบไฟล์ model
        model_path = os.path.join("public", "models", "handC", "model.json")
        weights_path = os.path.join("public", "models", "handC", "weights.bin")
        
        if os.path.exists(model_path) and os.path.exists(weights_path):
            try:
                # อ่าน config ของ model
                with open(model_path, 'r') as f:
                    model_config = json.load(f)
                
                # ดึงข้อมูลพื้นฐาน
                topology = model_config.get('modelTopology', {})
                architecture = topology.get('class_name', 'ไม่ทราบ')
                
                # ตรวจสอบขนาดไฟล์
                model_size = os.path.getsize(model_path)
                weights_size = os.path.getsize(weights_path)
                
                print(f"✅ ไฟล์ Model: พบครบทั้งคู่")
                print(f"📋 สถาปัตยกรรม: {architecture}")
                print(f"📊 ขนาด Model: {model_size:,} bytes")
                print(f"📊 ขนาด Weights: {weights_size:,} bytes")
                
                # จำลองการโหลด
                self.model_loaded = True
                print("✅ การโหลด Model: สำเร็จ (จำลอง)")
                
            except Exception as e:
                print(f"❌ การโหลด Model ผิดพลาด: {e}")
                self.model_loaded = False
        else:
            print("❌ ไฟล์ Model: ไฟล์หายไป")
            self.model_loaded = False
        
        # สรุปผล
        status = "พร้อมใช้งาน" if self.model_loaded else "ไม่พร้อมใช้งาน"
        print(f"\n📊 สถานะ HandC Model: {status}")

# ทดสอบการโหลด
handc_loader = HandCModelLoader()

print("\n🎯 การทดสอบโหลด Model เสร็จสิ้น!")
print("="*50)

⚙️ กำลังทดสอบการโหลด HandC Model...
🔍 กำลังตรวจสอบไฟล์ HandC Model...
✅ Metadata: พบคำศัพท์ 5 คำ
📝 คำศัพท์: ['ฉลาด', 'เป็นห่วง', 'ไม่สบาย', 'เข้าใจ', 'idle']
✅ ไฟล์ Model: พบครบทั้งคู่
📋 สถาปัตยกรรม: Sequential
📊 ขนาด Model: 1,435 bytes
📊 ขนาด Weights: 5,898,000 bytes
✅ การโหลด Model: สำเร็จ (จำลอง)

📊 สถานะ HandC Model: พร้อมใช้งาน

🎯 การทดสอบโหลด Model เสร็จสิ้น!


In [11]:
# 🎯 Cell 4: Model 2 - Face Detection (ทดสอบการโหลด Model)
print("⚙️ กำลังทดสอบการโหลด Face Detection Model...")

class FaceModelLoader:
    def __init__(self):
        self.models_found = {}
        self.model_loaded = False
        
        print("🔍 กำลังตรวจสอบไฟล์ Face Detection Model...")
        
        # ตรวจสอบโฟลเดอร์ face-models
        face_models_path = os.path.join("public", "face-models")
        
        if not os.path.exists(face_models_path):
            print("❌ ไฟล์ Face Models: ไม่พบโฟลเดอร์")
            return
        
        try:
            # สแกนไฟล์ในโฟลเดอร์
            files = os.listdir(face_models_path)
            print(f"📂 พบไฟล์ในโฟลเดอร์: {len(files)} ไฟล์")
            
            # ค้นหาไฟล์ model ที่สำคัญ (สำหรับ tiny face detector)
            required_files = [
                'tiny_face_detector_model-weights_manifest.json',
                'tiny_face_detector_model-shard1',
                'face_landmark_68_model-weights_manifest.json',
                'face_landmark_68_model-shard1',
                'face_expression_model-weights_manifest.json',
                'face_expression_model-shard1'
            ]
            
            found_models = []
            for file in files:
                for required in required_files:
                    if required in file:
                        found_models.append(file)
                        break
            
            print(f"📝 พบไฟล์ Model สำคัญ: {len(found_models)} ไฟล์")
            
            # เช็คขนาดไฟล์
            total_size = 0
            for file in files:
                try:
                    file_path = os.path.join(face_models_path, file)
                    if os.path.isfile(file_path):
                        size = os.path.getsize(file_path)
                        total_size += size
                        if file.endswith('.json') or 'shard' in file:
                            self.models_found[file] = f"{size:,} bytes"
                except:
                    continue
            
            print(f"📊 ขนาดรวม Face Models: {total_size:,} bytes")
            
            # แสดงรายละเอียดไฟล์สำคัญ
            if self.models_found:
                print("📋 รายละเอียดไฟล์:")
                for file, size in list(self.models_found.items())[:6]:  # แสดง 6 ไฟล์แรก
                    print(f"  • {file}: {size}")
                if len(self.models_found) > 6:
                    print(f"  • ... และอีก {len(self.models_found) - 6} ไฟล์")
            
            # ตรวจสอบไฟล์ที่จำเป็น (เปลี่ยนเป็น tiny face detector)
            has_tiny_face_detector = any('tiny_face_detector' in f for f in files)
            has_landmarks = any('landmark' in f for f in files)
            has_expressions = any('expression' in f for f in files)
            has_recognition = any('recognition' in f for f in files)
            
            print(f"✅ Tiny Face Detector: {'มี' if has_tiny_face_detector else 'ไม่มี'}")
            print(f"✅ Face Landmarks: {'มี' if has_landmarks else 'ไม่มี'}")
            print(f"✅ Face Expressions: {'มี' if has_expressions else 'ไม่มี'}")
            print(f"✅ Face Recognition: {'มี' if has_recognition else 'ไม่มี'}")
            
            # จำลองการโหลด (ต้องมี tiny face detector และ landmarks อย่างน้อย)
            if has_tiny_face_detector and has_landmarks:
                self.model_loaded = True
                print("✅ การโหลด Face Models: สำเร็จ (จำลอง)")
            else:
                print("❌ การโหลด Face Models: ไฟล์ไม่ครบ")
                
        except Exception as e:
            print(f"❌ Face Models ผิดพลาด: {e}")
            self.model_loaded = False
        
        # สรุปผล
        status = "พร้อมใช้งาน" if self.model_loaded else "ไม่พร้อมใช้งาน"
        print(f"\n📊 สถานะ Face Detection Model: {status}")

# ทดสอบการโหลด
face_loader = FaceModelLoader()

print("\n🎯 การทดสอบโหลด Face Model เสร็จสิ้น!")
print("="*50)

⚙️ กำลังทดสอบการโหลด Face Detection Model...
🔍 กำลังตรวจสอบไฟล์ Face Detection Model...
📂 พบไฟล์ในโฟลเดอร์: 12 ไฟล์
📝 พบไฟล์ Model สำคัญ: 6 ไฟล์
📊 ขนาดรวม Face Models: 7,403,868 bytes
📋 รายละเอียดไฟล์:
  • face_expression_model-shard1: 329,468 bytes
  • face_expression_model-weights_manifest.json: 6,384 bytes
  • face_landmark_68_model-shard1: 356,840 bytes
  • face_landmark_68_model-weights_manifest.json: 7,889 bytes
  • face_recognition_model-shard1: 4,194,304 bytes
  • face_recognition_model-shard2: 2,249,728 bytes
  • ... และอีก 3 ไฟล์
✅ Tiny Face Detector: มี
✅ Face Landmarks: มี
✅ Face Expressions: มี
✅ Face Recognition: มี
✅ การโหลด Face Models: สำเร็จ (จำลอง)

📊 สถานะ Face Detection Model: พร้อมใช้งาน

🎯 การทดสอบโหลด Face Model เสร็จสิ้น!


In [5]:
# Cell 5: Simple Image Capture Interface
print("Setting up Simple Image Capture Interface...")

import cv2
import numpy as np
import os
import glob
from datetime import datetime

class SimpleImageCapture:
    def __init__(self):
        self.image_counter = 1
        self.cap = None
        
    def start_camera(self):
        """Start camera"""
        try:
            self.cap = cv2.VideoCapture(0)
            if self.cap.isOpened():
                print("Camera started successfully!")
                return True
            else:
                print("Failed to start camera")
                return False
        except Exception as e:
            print(f"Camera error: {e}")
            return False
    
    def capture_and_save(self):
        """Capture image from camera and save as PNG"""
        if self.cap is None or not self.cap.isOpened():
            print("Camera not ready. Please start camera first.")
            return False
        
        try:
            ret, frame = self.cap.read()
            if ret:
                # Flip frame horizontally (mirror effect)
                frame = cv2.flip(frame, 1)
                
                # Create filename
                filename = f"test_image_{self.image_counter}.png"
                
                # Save image
                cv2.imwrite(filename, frame)
                
                print(f"Image saved: {filename}")
                print(f"Size: {frame.shape[1]}x{frame.shape[0]} pixels")
                
                self.image_counter += 1
                return True
            else:
                print("Failed to capture image")
                return False
                
        except Exception as e:
            print(f"Capture error: {e}")
            return False
    
    def list_saved_images(self):
        """List all saved test images"""
        saved_files = glob.glob("test_image_*.png")
        saved_files.sort()
        
        if saved_files:
            print(f"Saved images ({len(saved_files)} files):")
            for file in saved_files:
                size = os.path.getsize(file)
                print(f"  - {file} ({size:,} bytes)")
        else:
            print("No saved images found")
        
        return saved_files
    
    def clear_all_images(self):
        """Delete all test images"""
        try:
            files = glob.glob("test_image_*.png")
            deleted_count = 0
            
            for file in files:
                os.remove(file)
                deleted_count += 1
            
            self.image_counter = 1
            print(f"Deleted {deleted_count} images")
            return True
            
        except Exception as e:
            print(f"Delete error: {e}")
            return False
    
    def stop_camera(self):
        """Stop camera"""
        if self.cap is not None:
            self.cap.release()
            self.cap = None
            print("Camera stopped")

# Create capture instance
image_capture = SimpleImageCapture()

print("Simple Image Capture Interface ready!")
print("Usage:")
print("1. image_capture.start_camera() - Start camera")
print("2. image_capture.capture_and_save() - Capture and save image")
print("3. image_capture.list_saved_images() - List saved images")
print("4. image_capture.clear_all_images() - Delete all images")
print("5. image_capture.stop_camera() - Stop camera")
print("="*50)

Setting up Simple Image Capture Interface...
Simple Image Capture Interface ready!
Usage:
1. image_capture.start_camera() - Start camera
2. image_capture.capture_and_save() - Capture and save image
3. image_capture.list_saved_images() - List saved images
4. image_capture.clear_all_images() - Delete all images
5. image_capture.stop_camera() - Stop camera


In [6]:
# Cell 6: Camera Control and Image Capture
print("Starting camera and capturing images...")

# Start camera
print("1. Starting camera...")
if image_capture.start_camera():
    print("   Camera ready!")
    
    # Capture 3 images with user interaction
    print("\n2. Image capture (Press Enter to capture each image):")
    
    for i in range(3):
        input(f"   Press Enter to capture image {i+1}/3...")
        if image_capture.capture_and_save():
            print(f"   Image {i+1} captured successfully!")
        else:
            print(f"   Failed to capture image {i+1}")
    
    # Stop camera
    print("\n3. Stopping camera...")
    image_capture.stop_camera()
    
    # List saved images
    print("\n4. Saved images:")
    saved_files = image_capture.list_saved_images()
    
    if saved_files:
        print(f"\nSuccessfully captured {len(saved_files)} images!")
        print("Ready to test with models (run next cell)")
    else:
        print("No images were captured")
        
else:
    print("Failed to start camera. Please check camera connection.")

print("="*50)

Starting camera and capturing images...
1. Starting camera...
Camera started successfully!
   Camera ready!

2. Image capture (Press Enter to capture each image):
Camera started successfully!
   Camera ready!

2. Image capture (Press Enter to capture each image):
Image saved: test_image_1.png
Size: 640x480 pixels
   Image 1 captured successfully!
Image saved: test_image_1.png
Size: 640x480 pixels
   Image 1 captured successfully!
Image saved: test_image_2.png
Size: 640x480 pixels
   Image 2 captured successfully!
Image saved: test_image_2.png
Size: 640x480 pixels
   Image 2 captured successfully!
Image saved: test_image_3.png
Size: 640x480 pixels
   Image 3 captured successfully!

3. Stopping camera...
Image saved: test_image_3.png
Size: 640x480 pixels
   Image 3 captured successfully!

3. Stopping camera...
Camera stopped

4. Saved images:
Saved images (3 files):
  - test_image_1.png (397,861 bytes)
  - test_image_2.png (394,641 bytes)
  - test_image_3.png (392,664 bytes)

Successfull

In [12]:
# 🎯 Cell 7: Process Captured Images through Models (ทดสอบภาพผ่าน Models)
print("🧠 กำลังทดสอบภาพที่บันทึกผ่าน Hand Model และ Face Model...")

import cv2
import numpy as np
import glob
import os
from datetime import datetime

class ImageModelProcessor:
    def __init__(self):
        self.processed_results = []
    
    def load_saved_images(self):
        """โหลดภาพที่บันทึกไว้"""
        image_files = glob.glob("test_image_*.png")
        image_files.sort()  # เรียงตามชื่อไฟล์
        
        images_data = []
        for file_path in image_files:
            try:
                # โหลดภาพ
                image = cv2.imread(file_path)
                if image is not None:
                    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    images_data.append({
                        'filename': file_path,
                        'image': image_rgb,
                        'size': f"{image.shape[1]}x{image.shape[0]}",
                        'file_size': os.path.getsize(file_path)
                    })
                    print(f"✅ โหลดภาพ: {file_path} ({image.shape[1]}x{image.shape[0]})")
            except Exception as e:
                print(f"❌ โหลดภาพ {file_path} ผิดพลาด: {e}")
        
        return images_data
    
    def process_through_hand_model(self, image_data):
        """ทดสอบภาพผ่าน Hand Model"""
        try:
            # ตรวจสอบว่า Hand Model พร้อมใช้งานหรือไม่
            if not (hasattr(handc_loader, 'model_loaded') and handc_loader.model_loaded):
                return "Hand Model ไม่พร้อมใช้งาน"
            
            # จำลองการทำนาย Hand Gesture
            import random
            
            # ใช้คำศัพท์จริงจาก handc_loader
            if hasattr(handc_loader, 'vocabulary') and handc_loader.vocabulary:
                vocab = handc_loader.vocabulary
                predicted_word = random.choice(vocab)
                confidence = round(random.uniform(0.75, 0.98), 2)
                return f"'{predicted_word}' (confidence: {confidence})"
            else:
                return "คำศัพท์: 'เข้าใจ' (confidence: 0.85)"
                
        except Exception as e:
            return f"ข้อผิดพลาด: {e}"
    
    def process_through_face_model(self, image_data):
        """ทดสอบภาพผ่าน Face Model"""
        try:
            # ตรวจสอบว่า Face Model พร้อมใช้งานหรือไม่
            if not (hasattr(face_loader, 'model_loaded') and face_loader.model_loaded):
                return "Face Model ไม่พร้อมใช้งาน"
            
            # จำลองการทำนาย Face Detection & Emotion
            import random
            
            emotions = ['มีความสุข', 'เศร้า', 'โกรธ', 'ประหลาดใจ', 'กลัว', 'เฉยๆ']
            detected_faces = random.randint(0, 2)
            
            if detected_faces == 0:
                return "ไม่พบใบหน้า"
            elif detected_faces == 1:
                emotion = random.choice(emotions)
                confidence = round(random.uniform(0.70, 0.95), 2)
                return f"พบ 1 ใบหน้า - อารมณ์: {emotion} (confidence: {confidence})"
            else:
                return f"พบ {detected_faces} ใบหน้า"
                
        except Exception as e:
            return f"ข้อผิดพลาด: {e}"
    
    def process_all_images(self):
        """ประมวลผลภาพทั้งหมด"""
        print("🔍 กำลังค้นหาภาพที่บันทึกไว้...")
        
        # โหลดภาพทั้งหมด
        images_data = self.load_saved_images()
        
        if not images_data:
            print("❌ ไม่พบภาพที่บันทึกไว้")
            print("📋 กรุณารัน Cell 6 เพื่อจับภาพก่อน")
            return
        
        print(f"\n🧠 เริ่มประมวลผล {len(images_data)} ภาพ...")
        print("="*60)
        
        self.processed_results = []
        
        for i, img_data in enumerate(images_data, 1):
            print(f"\n📸 กำลังประมวลผล: {img_data['filename']}")
            print(f"   📏 ขนาด: {img_data['size']} pixels ({img_data['file_size']:,} bytes)")
            
            # ทดสอบผ่าน Hand Model
            print(f"   🤲 Hand Model: ", end="")
            hand_result = self.process_through_hand_model(img_data)
            print(hand_result)
            
            # ทดสอบผ่าน Face Model  
            print(f"   😊 Face Model: ", end="")
            face_result = self.process_through_face_model(img_data)
            print(face_result)
            
            # เก็บผลลัพธ์
            result = {
                'filename': img_data['filename'],
                'size': img_data['size'],
                'file_size': img_data['file_size'],
                'hand_result': hand_result,
                'face_result': face_result,
                'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            self.processed_results.append(result)
        
        print("\n" + "="*60)
        print(f"🎉 ประมวลผล {len(images_data)} ภาพเสร็จสิ้น!")
        
        return self.processed_results
    
    def show_summary(self):
        """แสดงสรุปผลลัพธ์"""
        if not self.processed_results:
            print("❌ ยังไม่มีผลลัพธ์การประมวลผล")
            return
        
        print("\n📊 สรุปผลการประมวลผล:")
        print("="*60)
        
        for i, result in enumerate(self.processed_results, 1):
            print(f"\n🖼️  ภาพที่ {i}: {result['filename']}")
            print(f"   📏 ขนาด: {result['size']} ({result['file_size']:,} bytes)")
            print(f"   🤲 Hand: {result['hand_result']}")
            print(f"   😊 Face: {result['face_result']}")
            print(f"   ⏰ เวลา: {result['timestamp']}")
        
        print("\n" + "="*60)
        
        # สถิติ
        total_images = len(self.processed_results)
        hand_success = sum(1 for r in self.processed_results if "confidence" in r['hand_result'])
        face_success = sum(1 for r in self.processed_results if "พบ" in r['face_result'])
        
        print(f"📈 สถิติ:")
        print(f"   📸 ภาพทั้งหมด: {total_images}")
        print(f"   🤲 Hand Detection สำเร็จ: {hand_success}/{total_images}")
        print(f"   😊 Face Detection สำเร็จ: {face_success}/{total_images}")

# สร้าง Image Processor
image_processor = ImageModelProcessor()

# เริ่มประมวลผลภาพ
results = image_processor.process_all_images()

# แสดงสรุปผลลัพธ์
image_processor.show_summary()

print("\n🎯 การทดสอบภาพผ่าน Models เสร็จสิ้น!")
print("="*50)

🧠 กำลังทดสอบภาพที่บันทึกผ่าน Hand Model และ Face Model...
🔍 กำลังค้นหาภาพที่บันทึกไว้...
✅ โหลดภาพ: test_image_1.png (640x480)
✅ โหลดภาพ: test_image_2.png (640x480)
✅ โหลดภาพ: test_image_3.png (640x480)

🧠 เริ่มประมวลผล 3 ภาพ...

📸 กำลังประมวลผล: test_image_1.png
   📏 ขนาด: 640x480 pixels (397,861 bytes)
   🤲 Hand Model: 'เป็นห่วง' (confidence: 0.94)
   😊 Face Model: พบ 2 ใบหน้า

📸 กำลังประมวลผล: test_image_2.png
   📏 ขนาด: 640x480 pixels (394,641 bytes)
   🤲 Hand Model: 'ไม่สบาย' (confidence: 0.82)
   😊 Face Model: พบ 1 ใบหน้า - อารมณ์: ประหลาดใจ (confidence: 0.87)

📸 กำลังประมวลผล: test_image_3.png
   📏 ขนาด: 640x480 pixels (392,664 bytes)
   🤲 Hand Model: 'เข้าใจ' (confidence: 0.92)
   😊 Face Model: พบ 2 ใบหน้า

🎉 ประมวลผล 3 ภาพเสร็จสิ้น!

📊 สรุปผลการประมวลผล:

🖼️  ภาพที่ 1: test_image_1.png
   📏 ขนาด: 640x480 (397,861 bytes)
   🤲 Hand: 'เป็นห่วง' (confidence: 0.94)
   😊 Face: พบ 2 ใบหน้า
   ⏰ เวลา: 2025-09-13 18:31:54

🖼️  ภาพที่ 2: test_image_2.png
   📏 ขนาด: 640x480 (394,641 byte

In [15]:
# 🎯 Cell 8: Typhoon API Integration (สร้างประโยคจาก Hand Gestures + Face Emotions)
print("🤖 กำลังส่งข้อมูลไปยัง Typhoon API...")

import json
import requests
import os

class TyphoonAPIProcessor:
    def __init__(self):
        self.api_url = "https://api.opentyphoon.ai/v1/chat/completions"
        self.api_key = None
        self.processed_sentences = []
        
        # ตรวจสอบ API Key
        self.load_api_key()
    
    def load_api_key(self):
        """โหลด API Key จากไฟล์ .env"""
        try:
            env_path = os.path.join("backend", ".env")
            if os.path.exists(env_path):
                with open(env_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        if line.startswith('TYPHOON_API_KEY='):
                            self.api_key = line.split('=', 1)[1].strip().strip('"\'')
                            print("✅ Typhoon API Key พบแล้ว")
                            return
                print("❌ ไม่พบ TYPHOON_API_KEY ในไฟล์ .env")
            else:
                print("❌ ไม่พบไฟล์ .env ใน backend/")
        except Exception as e:
            print(f"❌ ข้อผิดพลาดในการอ่าน API Key: {e}")
    
    def extract_data_from_results(self, results):
        """ดึงข้อมูล Hand gestures และ Face emotions จาก results"""
        extracted_data = {
            "hand_gestures": [],
            "face_emotions": [],
            "summary": {
                "total_images": len(results),
                "hand_words": [],
                "emotions": []
            }
        }
        
        for i, result in enumerate(results, 1):
            # ดึงคำจาก Hand Model
            hand_result = result.get('hand_result', '')
            hand_word = None
            if "'" in hand_result and "confidence" in hand_result:
                # Extract word from format like "'เป็นห่วง' (confidence: 0.94)"
                try:
                    hand_word = hand_result.split("'")[1]
                    extracted_data["hand_gestures"].append({
                        "image": i,
                        "word": hand_word,
                        "confidence": hand_result.split("confidence: ")[1].split(")")[0]
                    })
                    if hand_word not in extracted_data["summary"]["hand_words"]:
                        extracted_data["summary"]["hand_words"].append(hand_word)
                except:
                    pass
            
            # ดึงอารมณ์จาก Face Model
            face_result = result.get('face_result', '')
            emotion = None
            if "อารมณ์:" in face_result:
                try:
                    emotion = face_result.split("อารมณ์: ")[1].split(" (confidence")[0]
                    extracted_data["face_emotions"].append({
                        "image": i,
                        "emotion": emotion,
                        "confidence": face_result.split("confidence: ")[1].split(")")[0] if "confidence" in face_result else "N/A"
                    })
                    if emotion not in extracted_data["summary"]["emotions"]:
                        extracted_data["summary"]["emotions"].append(emotion)
                except:
                    pass
        
        return extracted_data
    
    def create_typhoon_request(self, hand_words, emotions):
        """สร้าง request สำหรับ Typhoon API"""
        
        # สร้าง JSON ของคำที่ได้
        words_data = {
            "hand_gestures": hand_words,
            "face_emotions": emotions
        }
        words_json = json.dumps(words_data, ensure_ascii=False, indent=2)
        
        system_prompt = """คุณเป็นผู้ช่วย AI ที่เชี่ยวชาญภาษาไทยและภาษามือไทย ให้สร้างประโยคไทยที่เป็นธรรมชาติ ถูกต้องตามหลักภาษา และใช้ในชีวิตประจำวันได้จริง"""

        user_prompt = f"""จากข้อมูลภาษามือไทยและอารมณ์เหล่านี้: {words_json}

กรุณาสร้างประโยคไทยที่เป็นธรรมชาติ 3 ประโยค โดย:
- ใช้คำภาษามือที่ให้มาทั้งหมดหรือส่วนใหญ่
- พิจารณาอารมณ์ที่ตรวจพบเพื่อให้ประโยคสอดคล้องกับบริบท
- เป็นประโยคที่คนไทยใช้จริงในชีวิตประจำวัน
- ถูกต้องตามหลักไวยากรณ์ไทย
- สื่อความหมายได้ชัดเจน
- เรียงคำต่อกันเป็นประโยคภาษาไทยที่สมบูรณ์ที่สุด

ตอบเป็นรายการประโยคเท่านั้น เช่น:
1. [ประโยคที่ 1]
2. [ประโยคที่ 2] 
3. [ประโยคที่ 3]"""

        return {
            "model": "typhoon-v2.1-12b-instruct",
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            "max_tokens": 1000,
            "temperature": 0.7,
            "top_p": 0.9
        }
    
    def call_typhoon_api(self, request_data):
        """เรียก Typhoon API"""
        if not self.api_key:
            return {"error": "API Key ไม่พร้อมใช้งาน"}
        
        try:
            headers = {
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            }
            
            print("📡 กำลังส่ง request ไปยัง Typhoon API...")
            response = requests.post(
                self.api_url, 
                headers=headers, 
                json=request_data,
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                return result
            else:
                return {
                    "error": f"API Error: {response.status_code}",
                    "details": response.text[:500]
                }
                
        except requests.exceptions.Timeout:
            return {"error": "API Timeout - ใช้เวลานานเกินไป"}
        except requests.exceptions.RequestException as e:
            return {"error": f"Request Error: {str(e)}"}
        except Exception as e:
            return {"error": f"Unexpected Error: {str(e)}"}
    
    def process_model_results(self, results):
        """ประมวลผลข้อมูลจาก Image Model และส่งไปยัง Typhoon API"""
        if not results:
            print("❌ ไม่มีข้อมูลผลลัพธ์จาก Image Models")
            return
        
        print("🔍 กำลังดึงข้อมูลจากผลลัพธ์การประมวลผลภาพ...")
        
        # ดึงข้อมูล
        extracted_data = self.extract_data_from_results(results)
        
        print(f"✅ พบคำภาษามือ: {extracted_data['summary']['hand_words']}")
        print(f"✅ พบอารมณ์: {extracted_data['summary']['emotions']}")
        
        if not extracted_data['summary']['hand_words'] and not extracted_data['summary']['emotions']:
            print("❌ ไม่พบข้อมูลที่ใช้สร้างประโยคได้")
            return
        
        # สร้าง request สำหรับ Typhoon API
        request_data = self.create_typhoon_request(
            extracted_data['summary']['hand_words'],
            extracted_data['summary']['emotions']
        )
        
        print(f"📋 ข้อมูลที่ส่งไป API:")
        print(f"   🤲 คำภาษามือ: {', '.join(extracted_data['summary']['hand_words'])}")
        print(f"   😊 อารมณ์: {', '.join(extracted_data['summary']['emotions'])}")
        
        # เรียก API
        api_result = self.call_typhoon_api(request_data)
        
        # แสดงผลลัพธ์
        self.display_typhoon_results(api_result, extracted_data)
        
        return api_result
    
    def display_typhoon_results(self, api_result, extracted_data):
        """แสดงผลลัพธ์จาก Typhoon API"""
        print("\n" + "="*60)
        print("🤖 ผลลัพธ์จาก Typhoon API:")
        print("="*60)
        
        if "error" in api_result:
            print(f"❌ ข้อผิดพลาด: {api_result['error']}")
            if "details" in api_result:
                print(f"📋 รายละเอียด: {api_result['details']}")
            
            # แสดงตัวอย่างประโยคแทน
            print("\n🔄 ตัวอย่างประโยคที่น่าจะได้:")
            sample_sentences = self.generate_sample_sentences(
                extracted_data['summary']['hand_words'],
                extracted_data['summary']['emotions']
            )
            for i, sentence in enumerate(sample_sentences, 1):
                print(f"   {i}. {sentence}")
        else:
            try:
                # ดึงข้อความจาก response
                content = api_result.get('choices', [{}])[0].get('message', {}).get('content', '')
                
                if content:
                    print("✅ ประโยคที่ Typhoon API สร้าง:")
                    print(content)
                    
                    # เก็บผลลัพธ์
                    self.processed_sentences.append({
                        "input_data": extracted_data,
                        "generated_sentences": content,
                        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    })
                else:
                    print("❌ ไม่พบเนื้อหาใน response")
                    
            except Exception as e:
                print(f"❌ ข้อผิดพลาดในการประมวลผล response: {e}")
        
        print("="*60)
    
    def generate_sample_sentences(self, hand_words, emotions):
        """สร้างประโยคตัวอย่างเมื่อ API ไม่ทำงาน"""
        samples = []
        
        if hand_words:
            if len(hand_words) >= 2:
                samples.append(f"เขา{hand_words[0]}และ{hand_words[1]}มาก")
                samples.append(f"ทำไมเธอถึง{hand_words[0]}แล้วก็{hand_words[1]}ด้วย")
            else:
                word = hand_words[0]
                samples.append(f"เขา{word}เรื่องนี้มาก")
                samples.append(f"อย่า{word}ไปเลยนะ")
            
            samples.append(f"การ{hand_words[0]}เป็นสิ่งดี")
        
        return samples[:3]

# ตรวจสอบว่ามีผลลัพธ์จาก Image Processing หรือไม่
if 'results' in locals() and results:
    print("🔍 พบข้อมูลผลลัพธ์จาก Image Models")
    
    # สร้าง Typhoon API Processor
    typhoon_processor = TyphoonAPIProcessor()
    
    # ประมวลผลและส่งไปยัง API
    api_response = typhoon_processor.process_model_results(results)
    
else:
    print("❌ ไม่พบข้อมูลผลลัพธ์จาก Image Models")
    print("📋 กรุณารัน Cell 7 ก่อนเพื่อประมวลผลภาพ")

print("\n🎯 การเชื่อมต่อ Typhoon API เสร็จสิ้น!")
print("="*50)

🤖 กำลังส่งข้อมูลไปยัง Typhoon API...
🔍 พบข้อมูลผลลัพธ์จาก Image Models
✅ Typhoon API Key พบแล้ว
🔍 กำลังดึงข้อมูลจากผลลัพธ์การประมวลผลภาพ...
✅ พบคำภาษามือ: ['เป็นห่วง', 'ไม่สบาย', 'เข้าใจ']
✅ พบอารมณ์: ['ประหลาดใจ']
📋 ข้อมูลที่ส่งไป API:
   🤲 คำภาษามือ: เป็นห่วง, ไม่สบาย, เข้าใจ
   😊 อารมณ์: ประหลาดใจ
📡 กำลังส่ง request ไปยัง Typhoon API...

🤖 ผลลัพธ์จาก Typhoon API:
✅ ประโยคที่ Typhoon API สร้าง:
1. ฉันเป็นห่วงเธอมากเลยนะ ไม่สบายหรือเปล่า ดูท่าทางประหลาดใจเหมือนไม่ค่อยรู้สึกตัวเลย
2. ไม่เป็นไรนะ เข้าใจแล้วว่าเธอคงเหนื่อย แต่ฉันยังเป็นห่วงเธออยู่ดี
3. เห็นท่าทางประหลาดใจแบบนี้ สงสัยเธอจะไม่สบายแน่ๆ เลย ฉันเป็นห่วงเธอจริงๆ นะ

🎯 การเชื่อมต่อ Typhoon API เสร็จสิ้น!

🤖 ผลลัพธ์จาก Typhoon API:
✅ ประโยคที่ Typhoon API สร้าง:
1. ฉันเป็นห่วงเธอมากเลยนะ ไม่สบายหรือเปล่า ดูท่าทางประหลาดใจเหมือนไม่ค่อยรู้สึกตัวเลย
2. ไม่เป็นไรนะ เข้าใจแล้วว่าเธอคงเหนื่อย แต่ฉันยังเป็นห่วงเธออยู่ดี
3. เห็นท่าทางประหลาดใจแบบนี้ สงสัยเธอจะไม่สบายแน่ๆ เลย ฉันเป็นห่วงเธอจริงๆ นะ

🎯 การเชื่อมต่อ Typhoon API เสร็จสิ้น!
